Understat Data for Teams-Players EPL (2014-2019)

In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
import pandas as pd
import requests
import os

def scrape_understat(payload):
    #Build request using url, headers (mimicking what Firefox does normally)
    #Works best with verify=True as you won't get the ssl errors. Payload is 
    #taylored for each request
    url = 'https://understat.com/main/getPlayersStats/'
    headers = {'content-type':'application/json; charset=utf-8',
    'Host': 'understat.com',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate, br',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Content-Length': '39',
    'Origin': 'https: // understat.com',
    'Connection': 'keep - alive',
    'Referer': 'https: // understat.com / league / EPL'
    }
    response = requests.post(url, data=payload, headers = headers, verify=True)
    response_json = response.json()
    inner_wrapper = response_json['response']
    json_player_data = inner_wrapper['players']
    return json_player_data

def clean_df(player_df, weeks):
    # Get rid of the columns that we don't care about
    #player_df.drop(['yellow_cards','red_cards', 'xGChain','xGBuildup','games','time'], axis=1, inplace=True)
    player_df  = player_df.rename(columns={'goals':'goals_'+weeks,'xG':'xG_'+weeks,'assists':'assists_'+weeks, 'xA':'xA_'+weeks, 'shots':'shots_'+weeks, 'key_passes':
        'key_passes_'+weeks,'npg':'npg_'+weeks,'npxG':'npxG_'+weeks})
    if weeks != '3wks':
        player_df.drop(['position','team_title'], axis=1, inplace=True)
    return(player_df)

#Create Pandas dataframes from each html table
print('Getting data for last 3 matches')
json_player_data = scrape_understat({'league':'EPL', 'season':'2019', 'n_last_matches': '3'})
three_game_table = pd.DataFrame(json_player_data)
three_game_df = clean_df(three_game_table,'3wks')
#Replace Position indentifiers with something more useful
three_game_df['position'] = three_game_df['position'].str.slice(0,1)
position_map = {'D':'DEF', 'F':'FWD', 'M':'MID', 'G':'GK', 'S':'FWD'}
three_game_df = three_game_df.replace({'position': position_map})

print('Getting data for last 5 matches')
json_player_data = scrape_understat({'league':'EPL', 'season':'2019', 'n_last_matches': '5'})
five_game_table = pd.DataFrame(json_player_data)
five_game_df = clean_df(five_game_table, '5wks')

print('Getting data for last 10 matches')
json_player_data = scrape_understat({'league':'EPL', 'season':'2019', 'n_last_matches': '10'})
ten_game_table = pd.DataFrame(json_player_data)
ten_game_df = clean_df(ten_game_table, '10wks')

print('Getting data for the whole season')
json_player_data = scrape_understat({'league':'EPL', 'season':'2019'})
season_table = pd.DataFrame(json_player_data)
season_df = clean_df(season_table, 'season')

print('Merging Tables')
EPL_player_df = pd.merge(three_game_df, five_game_df, on=['id','player_name'])
EPL_player_df = pd.merge(EPL_player_df, ten_game_df, on=['id','player_name'])
# EPL_player_df = pd.merge(EPL_player_df, season_df, on=['id','player_name'])

# print('Writing CSV File')
# EPL_player_df.to_csv('Understat_EPL_Player_Data_Combined.csv', encoding='utf-8', index=False)

Getting data for last 3 matches
Getting data for last 5 matches
Getting data for last 10 matches
Getting data for the whole season
Merging Tables


In [3]:
three_game_df

,id,player_name,games,time,goals_3wks,xG_3wks,assists_3wks,xA_3wks,shots_3wks,key_passes_3wks,yellow_cards,red_cards,position,team_title,npg_3wks,npxG_3wks,xGChain,xGBuildup
0,647,Harry Kane,3,270,5,2.0607955306768417,0,0.3910949155688286,7,2,1,0,FWD,Tottenham,5,2.0607955306768417,2.6767265051603317,0.22483597695827484
1,502,Olivier Giroud,3,234,3,3.1137402653694153,0,0.40248509496450424,12,2,1,0,FWD,Chelsea,3,3.1137402653694153,3.732365131378174,0.30892330408096313
2,618,Raheem Sterling,3,198,3,2.774513840675354,0,0.39271488785743713,11,3,0,0,FWD,Manchester City,3,2.013344943523407,2.352346047759056,0.712441697716713
3,986,Danny Ings,3,270,3,2.290885627269745,0,0.13838174007833004,10,3,0,0,FWD,Southampton,2,0.7685479409992695,1.814413771033287,1.169490970671177
4,4456,Chris Wood,3,264,3,2.1954740285873413,0,0.7095881886780262,10,4,0,0,FWD,Burnley,2,1.4343052208423615,1.5645041763782501,0.15562047436833382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,8493,Jake Vokins,1,90,0,0.013284161686897278,0,0.09659572690725327,1,1,0,0,DEF,Southampton,0,0.013284161686897278,0.5018613934516907,0.4885772168636322
511,8496,Tommy Doyle,1,13,0,0.07760051637887955,0,0,1,0,0,0,FWD,Manchester City,0,0.07760051637887955,0.07760051637887955,0
512,8518,Jordan Thomas,1,1,0,0,0,0,0,0,0,0,FWD,Norwich,0,0,0,0
513,8562,Luke Thomas,3,270,0,0,1,0.6299818009138107,0,9,0,0,DEF,Leicester,0,0,1.3788530230522156,1.2677359282970428


In [5]:
outname_3wks = 'last_3_gw-data.csv'
outdir_3wks = r'E:\\AMotefaker\\ABC\\FPA\\Web Scraping\\Understat Data for Teams-Players (2014- present)\\Understat Data Scraper\\Player_Data\\last_3_5_10_gw_data\\last_3_gw_data.csv'
if not os.path.exists(outdir_3wks):
    os.mkdir(outdir_3wks)
fullname_3wks = os.path.join(outname_3wks, outdir_3wks) 
three_game_df.to_csv('{}'.format(fullname_3wks),encoding = 'utf-8', index = False)

PermissionError: [Errno 13] Permission denied: 'E:\\\\AMotefaker\\\\ABC\\\\FPA\\\\Web Scraping\\\\Understat Data for Teams-Players (2014- present)\\\\Understat Data Scraper\\\\Player_Data\\\\last_3_5_10_gw_data\\\\last_3_gw_data.csv'

In [6]:
EPL_player_df[0:17]

,id,player_name,games_x,time_x,goals_3wks,xG_3wks,assists_3wks,xA_3wks,shots_3wks,key_passes_3wks,...,assists_10wks,xA_10wks,shots_10wks,key_passes_10wks,yellow_cards,red_cards,npg_10wks,npxG_10wks,xGChain,xGBuildup
0,647,Harry Kane,3,270,5,2.0607955306768417,0,0.3910949155688286,7,2,...,0,1.297498844563961,28,9,2,0,7,5.564089119434357,7.057452630251646,0.624919380992651
1,502,Olivier Giroud,3,234,3,3.1137402653694153,0,0.40248509496450424,12,2,...,0,0.5414802059531212,29,3,1,0,7,5.79319491237402,6.430627778172493,1.374771699309349
2,618,Raheem Sterling,3,198,3,2.774513840675354,0,0.39271488785743713,11,3,...,0,1.328054966405034,31,13,0,0,9,5.7317270040512085,7.737957417964935,2.165522690862417
3,986,Danny Ings,3,270,3,2.290885627269745,0,0.13838174007833004,10,3,...,1,1.0314845945686102,26,14,1,0,6,3.1079669073224068,6.144841525703669,2.6239782236516476
4,4456,Chris Wood,3,264,3,2.1954740285873413,0,0.7095881886780262,10,4,...,0,0.8576705902814865,21,10,0,0,3,3.677421435713768,3.3569836765527725,0.3470211364328861
5,7700,Che Adams,3,151,3,1.9055250883102417,0,0.41254836320877075,10,1,...,1,0.8078726157546043,21,4,0,0,4,3.1744526624679565,4.17007052898407,0.47648803144693375
6,318,Pierre-Emerick Aubameyang,3,210,2,1.455847904086113,1,0.3655194416642189,6,2,...,2,1.2012219205498695,28,6,0,0,4,3.5896055176854134,4.833086274564266,0.9512820690870285
7,447,Kevin De Bruyne,3,246,2,0.4195265993475914,2,2.7748354077339172,7,21,...,4,5.972339313477278,26,41,1,0,3,1.4963189624249935,9.413695573806763,3.9182050600647926
8,531,Michail Antonio,3,219,2,1.9048762023448944,0,0.08414974063634872,7,2,...,1,1.8581748697906733,36,13,3,0,7,8.16870960779488,8.764186352491379,1.3592907842248678
9,574,Troy Deeney,3,229,2,1.6963218748569489,0,0.45213213562965393,5,2,...,0,0.9918195381760597,14,10,0,0,1,1.6338527463376522,2.9426354561001062,1.566445479169488


In [7]:
EPL_player_df.columns

Index(['id', 'player_name', 'games_x', 'time_x', 'goals_3wks', 'xG_3wks',
       'assists_3wks', 'xA_3wks', 'shots_3wks', 'key_passes_3wks',
       'yellow_cards_x', 'red_cards_x', 'position', 'team_title', 'npg_3wks',
       'npxG_3wks', 'xGChain_x', 'xGBuildup_x', 'games_y', 'time_y',
       'goals_5wks', 'xG_5wks', 'assists_5wks', 'xA_5wks', 'shots_5wks',
       'key_passes_5wks', 'yellow_cards_y', 'red_cards_y', 'npg_5wks',
       'npxG_5wks', 'xGChain_y', 'xGBuildup_y', 'games', 'time', 'goals_10wks',
       'xG_10wks', 'assists_10wks', 'xA_10wks', 'shots_10wks',
       'key_passes_10wks', 'yellow_cards', 'red_cards', 'npg_10wks',
       'npxG_10wks', 'xGChain', 'xGBuildup'],
      dtype='object')

In [8]:
EPL_player_df.rename(columns = {'games_x' : 'games_in_3wks',
                                'time_x' :'games_in_3wks',
                                'yellow_cards_x': 'yellow_cards_3wks',
                                'red_cards_x' : 'red_cards_3wks',
                                'xGChain_x': 'xGChain_3wks',
                                'xGBuildup_x': 'xGBuildup_3wks', 
                                'games_y' : 'games_season', 
                                'time_y' : 'time_season',
                                'yellow_cards_y': 'yellow_cards_season',
                                'red_cards_y' : 'red_cards_season',
                                'xGChain_y': 'xGChain_season',
                                'xGBuildup_y': 'xGBuildup_season',
                                'games_x' : 'games_in_3wks',
                                'time_x' :'games_in_3wks',
                                'yellow_cards_x': 'yellow_cards_3wks',
                                'red_cards_x' : 'red_cards_3wks',
                                'xGChain_x': 'xGChain_3wks',
                                'xGBuildup_x': 'xGBuildup_3wks', 
                                'games_y' : 'games_season', 
                                'time_y' : 'time_season',
                                'yellow_cards_y': 'yellow_cards_season',
                                'red_cards_y' : 'red_cards_season',
                                'xGChain_y': 'xGChain_season',
                                'xGBuildup_y': 'xGBuildup_season'})

,id,player_name,games_in_3wks,games_in_3wks,goals_3wks,xG_3wks,assists_3wks,xA_3wks,shots_3wks,key_passes_3wks,...,assists_10wks,xA_10wks,shots_10wks,key_passes_10wks,yellow_cards,red_cards,npg_10wks,npxG_10wks,xGChain,xGBuildup
0,647,Harry Kane,3,270,5,2.0607955306768417,0,0.3910949155688286,7,2,...,0,1.297498844563961,28,9,2,0,7,5.564089119434357,7.057452630251646,0.624919380992651
1,502,Olivier Giroud,3,234,3,3.1137402653694153,0,0.40248509496450424,12,2,...,0,0.5414802059531212,29,3,1,0,7,5.79319491237402,6.430627778172493,1.374771699309349
2,618,Raheem Sterling,3,198,3,2.774513840675354,0,0.39271488785743713,11,3,...,0,1.328054966405034,31,13,0,0,9,5.7317270040512085,7.737957417964935,2.165522690862417
3,986,Danny Ings,3,270,3,2.290885627269745,0,0.13838174007833004,10,3,...,1,1.0314845945686102,26,14,1,0,6,3.1079669073224068,6.144841525703669,2.6239782236516476
4,4456,Chris Wood,3,264,3,2.1954740285873413,0,0.7095881886780262,10,4,...,0,0.8576705902814865,21,10,0,0,3,3.677421435713768,3.3569836765527725,0.3470211364328861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,8493,Jake Vokins,1,90,0,0.013284161686897278,0,0.09659572690725327,1,1,...,0,0.09659572690725327,1,1,0,0,0,0.013284161686897278,0.5018613934516907,0.4885772168636322
511,8496,Tommy Doyle,1,13,0,0.07760051637887955,0,0,1,0,...,0,0,1,0,0,0,0,0.07760051637887955,0.07760051637887955,0
512,8518,Jordan Thomas,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
513,8562,Luke Thomas,3,270,0,0,1,0.6299818009138107,0,9,...,1,0.6299818009138107,0,9,0,0,0,0,1.3788530230522156,1.2677359282970428


In [9]:
import requests
from bs4 import BeautifulSoup 
import json
import pandas as pd

In [10]:
base_url = 'https://understat.com/league/EPL/'
season_names = ['2014', '2015', '2016', '2017', '2018','2019', '2020']

Getting data for EPL 2014

In [11]:
url = base_url + season_names[0]
res = requests.get(url)
soup = BeautifulSoup(res.content, 'lxml')

In [12]:
# data is under "scripts" tag. So, finding all the script tags
script = soup.find_all('script')
# print(script)

In [13]:
string_with_json_obj = ''

for el in script:
    if 'teamsData' in el.text:
        string_with_json_obj = el.text.strip()
# print(string_with_json_obj)

In [14]:
#keeping only the part of string we need
ind_start = string_with_json_obj.index("('") + 2
ind_end = string_with_json_obj.index("')") 
json_data = string_with_json_obj[ind_start: ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

In [16]:
print(json.dumps(data, indent=4, sort_keys=True))

NameError: name 'data' is not defined

In [17]:
data = json.loads(json_data)
print(data.keys())
# print('-' * 100)
# print(data['71'].keys())
# print('-' * 100)
# print(data['71']['id'])
# print('-' * 100)
# print(data['71']['title'])
# print('-' * 100)
# print(data['71']['history'])

dict_keys(['71', '72', '74', '75', '76', '77', '78', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '91', '92', '202'])


Making a dictionary composed opf team ID and team titles

In [18]:
teams ={}

for id in data.keys():
    teams[id] = data[id]['title']

In [19]:
teams

{'71': 'Aston Villa',
 '72': 'Everton',
 '74': 'Southampton',
 '75': 'Leicester',
 '76': 'West Bromwich Albion',
 '77': 'Sunderland',
 '78': 'Crystal Palace',
 '80': 'Chelsea',
 '81': 'West Ham',
 '82': 'Tottenham',
 '83': 'Arsenal',
 '84': 'Swansea',
 '85': 'Stoke',
 '86': 'Newcastle United',
 '87': 'Liverpool',
 '88': 'Manchester City',
 '89': 'Manchester United',
 '91': 'Hull',
 '92': 'Burnley',
 '202': 'Queens Park Rangers'}

In [20]:
columns = []
values = []
for id in data.keys():
    columns = list(data[id]['history'][0].keys())
    values = list(data[id]['history'][0].values())
    break

print(columns)
# print(values)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']


In [21]:
# QPR_data = []

# for i in data['202']['history']:
#     QPR_data.append(list(i.values()))
    
# print(QPR_data)  

In [22]:
# s = json.dumps(data, indent = 2, sort_keys =True)
# print(s)

In [23]:
# print(teams.items())
# type(data['202']['history'][0].values())

In [24]:
all_team_dicts = {}

for id, team in teams.items():
    teams_data = []
    for i in data[id]['history']:
        teams_data.append(list(i.values()))
        
    df = pd.DataFrame(teams_data, columns = columns)
    all_team_dicts[team] = df
    print('Added data for {}.'.format(team))

Added data for Aston Villa.
Added data for Everton.
Added data for Southampton.
Added data for Leicester.
Added data for West Bromwich Albion.
Added data for Sunderland.
Added data for Crystal Palace.
Added data for Chelsea.
Added data for West Ham.
Added data for Tottenham.
Added data for Arsenal.
Added data for Swansea.
Added data for Stoke.
Added data for Newcastle United.
Added data for Liverpool.
Added data for Manchester City.
Added data for Manchester United.
Added data for Hull.
Added data for Burnley.
Added data for Queens Park Rangers.


In [25]:
all_team_dicts['Manchester United']

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,h,1.166350,0.278076,1.166350,0.278076,"{'att': 253, 'def': 25}","{'att': 341, 'def': 19}",8,2,1,2,2.2359,l,2014-08-16 12:45:00,0,0,1,0,0.888274
1,a,0.941557,0.666489,0.941557,0.666489,"{'att': 297, 'def': 27}","{'att': 267, 'def': 21}",2,4,1,1,1.6740,d,2014-08-24 16:00:00,0,1,0,1,0.275068
2,a,0.467281,0.513746,0.467281,0.513746,"{'att': 150, 'def': 21}","{'att': 305, 'def': 27}",11,2,0,0,1.2096,d,2014-08-30 12:45:00,0,1,0,1,-0.046465
3,h,2.209110,0.563524,2.209110,0.563524,"{'att': 228, 'def': 26}","{'att': 546, 'def': 12}",9,4,4,0,2.5363,w,2014-09-14 16:00:00,1,0,0,3,1.645586
4,a,1.711910,2.542200,1.711910,1.019860,"{'att': 174, 'def': 26}","{'att': 285, 'def': 23}",7,6,3,5,0.8067,l,2014-09-21 13:30:00,0,0,1,0,0.692050
5,h,0.520246,1.274520,0.520246,1.274520,"{'att': 189, 'def': 13}","{'att': 315, 'def': 21}",5,10,2,1,0.6965,w,2014-09-27 15:00:00,1,0,0,3,-0.754274
6,h,1.623900,2.135580,1.623900,1.374410,"{'att': 268, 'def': 36}","{'att': 269, 'def': 20}",4,7,2,1,0.9917,w,2014-10-05 12:00:00,1,0,0,3,0.249490
7,a,0.934042,0.616620,0.934042,0.616620,"{'att': 214, 'def': 27}","{'att': 286, 'def': 16}",8,3,2,2,1.5720,d,2014-10-20 20:00:00,0,1,0,1,0.317422
8,h,1.840740,0.989916,1.840740,0.989916,"{'att': 245, 'def': 18}","{'att': 254, 'def': 18}",7,8,1,1,2.0214,d,2014-10-26 16:00:00,0,1,0,1,0.850824
9,a,0.727924,2.567230,0.727924,2.567230,"{'att': 253, 'def': 19}","{'att': 258, 'def': 35}",5,14,0,1,0.3017,l,2014-11-02 13:30:00,0,0,1,0,-1.839306


In [27]:
import os

for team, df in all_team_dicts.items():
    df['ppda_coef'] =  df['ppda'].apply(lambda x : x['att']/x['def'] if x['def'] != 0 else 0)
    df['oppda_coef'] = df['ppda_allowed'].apply(lambda x : x['att']/x['def'] if x['def'] != 0 else 0)
    df['GW'] = range(1,len(df) + 1)
    df = df[['GW','date','result','h_a','xG','scored', 'xGA', 'missed', 'xpts', 'pts', 'npxG', 'npxGA','ppda', 'ppda_allowed', 'ppda_coef', 'oppda_coef', 'deep',
       'deep_allowed', 'npxGD']]
    df.rename(columns = {'missed': 'conceded',
                        'h_a': 'Home/Away'}, inplace = True)
    outname = '{}_{}_season data.csv'.format(team, season_names[0])
    outdir = r'E:\AMotefaker\ABC\FPA\Web Scraping\Understat Data for Teams-Players (2014- present)\Understat Data Scraper\Team_Data\{}_season'.format(season_names[0])
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    fullname = os.path.join(outdir, outname) 
    df.to_csv('{}'.format(fullname), index = False)

C:\Users\Yepco\AppData\Local\Temp\ipykernel_12016\2740325498.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'missed': 'conceded',
C:\Users\Yepco\AppData\Local\Temp\ipykernel_12016\2740325498.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'missed': 'conceded',
C:\Users\Yepco\AppData\Local\Temp\ipykernel_12016\2740325498.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(colu

In [28]:
outname = '{}_{}_season data.csv'.format(team, season_names[0])
outdir = r'E:\AMotefaker\ABC\FPA\Web Scraping\Understat Data for Teams-Players (2014- present)\Understat Data Scraper\Team_Data\{}_season'.format(season_names[0])

print(outname + outdir)

Queens Park Rangers_2014_season data.csvE:\AMotefaker\ABC\FPA\Web Scraping\Understat Data for Teams-Players (2014- present)\Understat Data Scraper\Team_Data\2014_season


In [29]:
all_team_dicts['Manchester United'].columns

Index(['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep',
       'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins',
       'draws', 'loses', 'pts', 'npxGD', 'ppda_coef', 'oppda_coef', 'GW'],
      dtype='object')

Correcting the format of PPDA and PPDA(PPDA allowed) to the ratio of PPDA and PPDA allowed

In [30]:
for team, df in all_team_dicts.items():
    all_team_dicts[team]['ppda_coef'] = all_team_dicts[team]['ppda'].apply(lambda x : x['att']/x['def'] if x['def'] != 0 else 0)
    all_team_dicts[team]['oppda_coef'] = all_team_dicts[team]['ppda_allowed'].apply(lambda x : x['att']/x['def'] if x['def'] != 0 else 0)

Getting columns to be averaged and summed

In [31]:
cols_to_sum = ['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'wins', 'draws', 'loses', 'pts', 'npxGD']
cols_to_mean = ['ppda_coef', 'oppda_coef']

In [32]:
frames = []

for team, df in all_team_dicts.items():
    sum_data = pd.DataFrame(df[cols_to_sum].sum()).transpose() # returns series data, so used transpose() 
    mean_data = pd.DataFrame(df[cols_to_mean].mean()).transpose() # returns series data, so used transpose() 
    sum_mean_data = sum_data.join(mean_data)
    sum_mean_data['team'] = team
    sum_mean_data['matches'] = len(df)
    frames.append(sum_mean_data)
    
final_data = pd.concat(frames, sort = False, ignore_index =True)

In [33]:
final_data = final_data[['team', 'matches', 'wins', 'draws', 'loses', 'scored', 'missed', 'pts','xG', 'npxG', 'xGA', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts']]
final_data.sort_values('pts', ascending= False, inplace = True)
final_data.reset_index(inplace = True, drop = True)
final_data['position'] = range(1, len(final_data) + 1)

Finding the difference between "expected" and "real" values

In [34]:
final_data.rename(columns ={'missed': 'conceded'}, inplace = True) # renamin 'missed' column to 'conceded'
final_data['xG_diff'] = final_data['xG'] - final_data['scored']
final_data['xGA_diff'] = final_data['xGA'] - final_data['conceded']
final_data['xpts_diff'] = final_data['xpts'] - final_data['pts']  

Converting appropriate columns to integer values

In [35]:
cols_to_int = ['wins', 'draws', 'loses', 'scored', 'conceded', 'pts', 'deep', 'deep_allowed']
final_data[cols_to_int] = final_data[cols_to_int].astype(int)

Formatting and changing the order of the dataframe

In [36]:
col_order = ['position','team', 'matches', 'wins', 'draws', 'loses', 'scored', 'conceded', 'pts', 'xG', 'xG_diff', 'npxG', 'xGA', 'xGA_diff', 'npxGA', 'npxGD', 'ppda_coef', 'oppda_coef', 'deep', 'deep_allowed', 'xpts', 'xpts_diff']
final_data = final_data[col_order]
pd.options.display.float_format = '{:,.2f}'.format

In [37]:
final_data

,position,team,matches,wins,draws,loses,scored,conceded,pts,xG,...,xGA,xGA_diff,npxGA,npxGD,ppda_coef,oppda_coef,deep,deep_allowed,xpts,xpts_diff
0,1,Chelsea,38,26,9,3,73,32,87,68.64,...,31.52,-0.48,29.24,35.50,10.94,13.42,407,171,75.32,-11.68
1,2,Manchester City,38,24,7,7,83,38,79,75.82,...,40.50,2.50,37.45,32.15,7.98,15.08,575,144,73.10,-5.90
2,3,Arsenal,38,22,9,7,71,36,75,69.80,...,35.72,-0.28,33.44,31.04,8.66,13.25,398,171,75.17,0.17
3,4,Manchester United,38,20,10,8,62,37,70,54.21,...,39.84,2.84,36.80,13.60,7.65,15.52,267,194,63.03,-6.97
4,5,Tottenham,38,19,7,12,58,53,64,52.39,...,57.04,4.04,51.60,-3.17,8.00,11.30,210,232,48.94,-15.06
5,6,Liverpool,38,18,8,12,52,48,62,51.70,...,38.25,-9.75,35.83,11.30,9.91,12.12,306,201,61.17,-0.83
6,7,Southampton,38,18,6,14,54,33,60,54.97,...,39.11,6.11,37.59,15.09,9.26,14.88,270,183,62.82,2.82
7,8,Swansea,38,16,8,14,46,49,56,40.90,...,55.94,6.94,54.42,-15.04,12.83,11.46,156,310,43.32,-12.68
8,9,Stoke,38,15,9,14,48,45,54,46.26,...,47.03,2.03,43.22,0.75,10.41,10.60,195,239,51.97,-2.03
9,10,Crystal Palace,38,13,9,16,47,51,48,44.76,...,45.48,-5.52,40.82,-0.77,11.58,6.58,180,258,50.56,2.56


Importing players data

In [1]:
import requests
import json
import pandas as pd
import os

In [2]:
season_names = ['2014', '2015', '2016', '2017', '2018', '2019']
gws = ['3', '5', '10']
leagues = ['EPL', 'La_liga', 'Bundesliga', 'Serie_A', 'Ligue_1']

In [3]:
def scrape_understat(payload):
    #Build request using url, headers (mimicking what Firefox does normally)
    #Works best with verify=True as you won't get the ssl errors. Payload is 
    #taylored for each request
    url = 'https://understat.com/main/getPlayersStats/'
    headers = {'content-type':'application/json; charset=utf-8',
    'Host': 'understat.com',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate, br',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'X-Requested-With': 'XMLHttpRequest',
    'Content-Length': '39',
    'Origin': 'https: // understat.com',
    'Connection': 'keep - alive',
    'Referer': 'https: // understat.com / league / EPL'
    }
    response = requests.post(url, data=payload, headers = headers, verify=True)
    response_json = response.json()
    inner_wrapper = response_json['response']
    json_player_data = inner_wrapper['players']
    return json_player_data

In [4]:
def clean_df(player_df, weeks):
    # Get rid of the columns that we don't care about
    #player_df.drop(['yellow_cards','red_cards', 'xGChain','xGBuildup','games','time'], axis=1, inplace=True)
    player_df  = player_df.rename(columns={'goals':'goals_'+weeks,'xG':'xG_'+weeks,'assists':'assists_'+weeks, 'xA':'xA_'+weeks, 'shots':'shots_'+weeks, 'key_passes':
        'key_passes_'+weeks,'npg':'npg_'+weeks,'npxG':'npxG_'+weeks})
    
    return(player_df)

In [5]:
def gw_data(season , league,  no_of_gw):
#     Create Pandas dataframes from each html table
    print('Getting data for last {} matches'.format(no_of_gw))
    json_player_data = scrape_understat({'league':'EPL', 'season':season, 'n_last_matches': no_of_gw})
    gw_table = pd.DataFrame(json_player_data)
    gw_df = clean_df(gw_table,'3wks')
    #Replace Position indentifiers with something more useful
    gw_df['position'] = gw_df['position'].str.slice(0,1)
    position_map = {'D':'DEF', 'F':'FWD', 'M':'MID', 'G':'GK', 'S':'FWD'}
    gw_df = gw_df.replace({'position': position_map})
    gw_df.to_csv(r'E:\AMotefaker\ABC\FPA\Web Scraping\Understat Data for Teams-Players (2014- present)\Understat Data Scraper\Player_Data\gw_data\last_{}_gw_data.csv'.format(no_of_gw), encoding='utf-8', index=False)
    print('last {} matches csv data written'.format(no_of_gw))
    return gw_df

In [6]:
last_3_gw_data_EPL = gw_data(season_names[-1], leagues[0], gws[0])
last_5_gw_data_EPL = gw_data(season_names[-1], leagues[0], gws[1])
last_10_gw_data_EPL = gw_data(season_names[-1], leagues[0], gws[2])

Getting data for last 3 matches
last 3 matches csv data written
Getting data for last 5 matches
last 5 matches csv data written
Getting data for last 10 matches
last 10 matches csv data written


In [11]:
def season_data(season, league):
    print('Getting data for {} season'.format(season))
    json_player_data = scrape_understat({'league': league, 'season':season})
    season_table = pd.DataFrame(json_player_data)
    season_df = clean_df(season_table, 'season')
    season_df.to_csv(r'E:\AMotefaker\ABC\FPA\Web Scraping\Understat Data for Teams-Players (2014- present)\Understat Data Scraper\Player_Data\season_data\{}_whole_season_data.csv'.format(season), encoding='utf-8', index=False)
    print('csv file for {} season written'.format(season))
    return season_df

In [12]:
season_1415 = season_data(season_names[0], leagues[0])
season_1516 = season_data(season_names[1], leagues[0])
season_1617 = season_data(season_names[2], leagues[0])
season_1718 = season_data(season_names[3], leagues[0])
season_1819 = season_data(season_names[4], leagues[0])
season_1920 = season_data(season_names[5], leagues[0])

Getting data for 2014 season
csv file for 2014 season written
Getting data for 2015 season
csv file for 2015 season written
Getting data for 2016 season
csv file for 2016 season written
Getting data for 2017 season
csv file for 2017 season written
Getting data for 2018 season
csv file for 2018 season written
Getting data for 2019 season
csv file for 2019 season written


In [16]:
season_df

NameError: name 'season_df' is not defined

In [17]:
season_1415

,id,player_name,games,time,goals_season,xG_season,assists_season,xA_season,shots_season,key_passes_season,yellow_cards,red_cards,position,team_title,npg_season,npxG_season,xGChain,xGBuildup
0,619,Sergio Agüero,33,2551,26,25.270159743726254,8,5.568922242149711,148,33,4,0,F S,Manchester City,21,20.70318364351988,27.805154908448458,6.878173082135618
1,647,Harry Kane,34,2589,21,17.15729223564267,4,3.922500966116786,112,27,4,0,F M S,Tottenham,19,14.873822528868914,16.488438992761075,5.549698735587299
2,802,Diego Costa,26,2111,20,15.219103761017323,3,4.554670915938914,76,41,8,0,F S,Chelsea,19,14.45793492347002,21.365789908915758,5.2769727278500795
3,848,Charlie Austin,35,3078,18,17.881850190460682,5,2.548747032880783,131,23,4,1,F,Queens Park Rangers,15,14.076042897999287,13.718276167288423,3.041321298107505
4,498,Alexis Sánchez,35,2967,16,13.451749975793064,8,8.49417957291007,122,82,4,0,F M S,Arsenal,16,12.690581078641117,27.15757153555751,10.736752955242991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,4528,Alejandro Faurlin,2,94,0,0.11255800724029541,0,0.4859839975833893,3,1,0,0,M S,Queens Park Rangers,0,0.11255800724029541,0.055191781371831894,0.022535664960741997
527,4530,Valentin Roberge,1,90,0,0.2825593948364258,0,0,1,0,1,0,D,Sunderland,0,0.2825593948364258,0,0
528,4531,Stuart O&#039;Keefe,2,88,0,0,0,0.019946111366152763,0,1,0,0,M S,Crystal Palace,0,0,0.019946111366152763,0.019946111366152763
529,4532,Gary Taylor-Fletcher,1,16,0,0,0,0,0,0,0,0,S,Leicester,0,0,0,0


In [18]:
season_1516

,id,player_name,games,time,goals_season,xG_season,assists_season,xA_season,shots_season,key_passes_season,yellow_cards,red_cards,position,team_title,npg_season,npxG_season,xGChain,xGBuildup
0,647,Harry Kane,38,3382,25,22.732073578983545,1,3.088511780835688,159,44,5,0,F,Tottenham,20,18.926266126334667,26.939671490341425,8.189033068716526
1,619,Sergio Agüero,30,2399,24,20.077213659882545,2,2.0287597198039293,119,27,1,0,F S,Manchester City,20,16.271369472146034,18.786789989098907,3.925958522595465
2,755,Jamie Vardy,36,3154,24,22.135407269001007,6,6.267152491956949,115,48,5,1,F,Leicester,19,17.56843115389347,24.595971267670393,2.8863560035824776
3,594,Romelu Lukaku,37,3194,18,20.05878111720085,6,5.6804345063865185,118,51,3,0,F S,Everton,17,18.53648015856743,24.024903159588575,7.63484107516706
4,750,Riyad Mahrez,37,3087,17,11.882148338481784,11,11.45409781858325,86,68,1,0,M S,Leicester,13,7.315135253593326,21.18506269901991,7.174407683312893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,1078,Sergi Canos,1,7,0,0,0,0,0,0,0,0,S,Liverpool,0,0,0,0
546,1079,Thomas Robson,1,90,0,0,0,0,0,0,0,0,D,Sunderland,0,0,0.09102849662303925,0.09102849662303925
547,1080,Rees Greenwood,1,53,0,0,0,0.02298414707183838,0,1,0,0,M,Sunderland,0,0,0.6708943247795105,0.6708943247795105
548,1084,Jonjoe Kenny,1,62,0,0.09364968538284302,0,0,1,0,0,0,S,Everton,0,0.09364968538284302,0.09364968538284302,0


In [19]:
season_1617

,id,player_name,games,time,goals_season,xG_season,assists_season,xA_season,shots_season,key_passes_season,yellow_cards,red_cards,position,team_title,npg_season,npxG_season,xGChain,xGBuildup
0,647,Harry Kane,30,2556,29,19.82009919732809,7,5.5538915153592825,110,41,3,0,F M S,Tottenham,24,15.253085978329182,21.94719305820763,4.12599990144372
1,594,Romelu Lukaku,37,3271,25,16.665452419780195,6,5.440816408023238,110,47,3,0,F S,Everton,24,15.904283582232893,21.251998490653932,3.9702013842761517
2,498,Alexis Sánchez,38,3233,24,17.724449135363102,10,8.988117071799934,129,78,6,0,F M S,Arsenal,22,15.44094256311655,27.14118772558868,9.08672859147191
3,619,Sergio Agüero,31,2408,20,22.672585003077984,3,4.665906261652708,139,31,4,1,F S,Manchester City,16,18.866740860044956,25.35206551104784,5.0266740852966905
4,802,Diego Costa,35,3101,20,15.431797001510859,7,4.912058966234326,111,42,10,0,F,Chelsea,20,14.670628163963556,22.003214471042156,6.565549122169614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,5575,Joel Pereira,1,90,0,0,0,0,0,0,0,0,GK,Manchester United,0,0,0.2117270529270172,0.2117270529270172
520,5576,James Husband,1,62,0,0,0,0,0,0,0,0,D,Middlesbrough,0,0,0.15693369507789612,0.15693369507789612
521,5586,Michael Folivi,1,6,0,0,0,0,0,0,0,0,S,Watford,0,0,0,0
522,5598,Angel Gomes,1,1,0,0,0,0,0,0,0,0,S,Manchester United,0,0,0,0
